In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [30]:
def pad_to_length(tensor_list, max_len=7):
    padded = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    if padded.size(1) < max_len:
        pad_size = max_len - padded.size(1)
        padded = F.pad(padded, (0, pad_size), value=0)
    else:
        padded = padded[:, :max_len]
    return padded

# Function to process and pad .npy dataset
def process_round_data(npy_file, max_len=7):
    data = np.load(npy_file)
    x = data[:, :-1]
    y = data[:, -1]
    x_tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in x]
    x_padded = pad_to_length(x_tensor_list, max_len=max_len)
    y_tensor = torch.tensor(y, dtype=torch.float)
    return x_padded, y_tensor

# === Load and process each round ===
x_first, y_first = process_round_data('data_for_first_round.npy', max_len=7)
x_second, y_second = process_round_data('data_for_second_round.npy', max_len=7)
x_third, y_third = process_round_data('data_for_third_round.npy', max_len=7)

# === Concatenate all rounds and repeat them===
x_first_repeated = x_first.repeat((20, 1))        
y_first_repeated = y_first.repeat((20,)) 
x_second_repeated = x_second.repeat((5, 1))        
y_second_repeated = y_second.repeat((5,))           

x_all = torch.cat([x_first_repeated, x_second_repeated, x_third], dim=0)
y_all = torch.cat([y_first_repeated, y_second_repeated, y_third], dim=0)

# === Final dataset and dataloader ===
dataset_all = TensorDataset(x_all, y_all)
dataloader_all = DataLoader(dataset_all, batch_size=32, shuffle=True)

# === Inspect sizes ===
print("First round input shape:", x_first.shape)
print("First round label shape:", y_first.shape)

print("First round repeated input shape:", x_first_repeated.shape)
print("First round repeated label shape:", y_first_repeated.shape)

print("Second round input shape:", x_second.shape)
print("Second round label shape:", y_second.shape)

print("Second round repeated input shape:", x_second_repeated.shape)
print("Second round repeated label shape:", y_second_repeated.shape)

print("Third round input shape:", x_third.shape)
print("Third round label shape:", y_third.shape)

print("Combined input shape:", x_all.shape)
print("Combined label shape:", y_all.shape)

First round input shape: torch.Size([1411, 7])
First round label shape: torch.Size([1411])
First round repeated input shape: torch.Size([28220, 7])
First round repeated label shape: torch.Size([28220])
Second round input shape: torch.Size([39996, 7])
Second round label shape: torch.Size([39996])
Second round repeated input shape: torch.Size([199980, 7])
Second round repeated label shape: torch.Size([199980])
Third round input shape: torch.Size([598017, 7])
Third round label shape: torch.Size([598017])
Combined input shape: torch.Size([826217, 7])
Combined label shape: torch.Size([826217])


## MODEL

In [33]:
class EmbeddingNetLinear(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetLinear, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.fc1 = nn.Linear(num_cards * embed_dim, 128)  # Increased capacity
        self.fc2 = nn.Linear(128, 64)                     # New layer
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)                       # Output layer

    def forward(self, x):
        embedded = self.embedding(x)  # Shape: [batch_size, num_cards, embed_dim]
        flat = embedded.view(x.size(0), self.num_cards * self.embed_dim)
        out = F.relu(self.fc1(flat))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        return torch.sigmoid(self.fc4(out)).squeeze(1)
    

class EmbeddingNetConv1D(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetConv1D, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=16, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4)

        self.pool = nn.AdaptiveMaxPool1d(1)  # Reduces to [batch, 64, 1]

        self.fc = nn.Linear(64, 1)  # Final output layer

    def forward(self, x):
        x = self.embedding(x)           # [batch, 7, embed_dim]
        x = x.transpose(1, 2)           # [batch, embed_dim, 7]
        x = F.relu(self.conv1(x))       # [batch, 32, 6]
        x = F.relu(self.conv2(x))       # [batch, 64, 4]
        x = F.relu(self.conv3(x))       # [batch, 64, 1]
        x = self.pool(x).squeeze(-1)    # [batch, 64]
        return torch.sigmoid(self.fc(x)).squeeze(1)

## TRAIN FUNCTION

In [ ]:
def train_model(model, dataloader, epochs=10, lr=0.001):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # Because we're using sigmoid in the model

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for x_batch, y_batch in dataloader:
            x_batch = x_batch
            y_batch = y_batch

            optimizer.zero_grad()
            preds = model(x_batch)

            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = (preds > 0.5).float()
            correct += (predicted in [y_batch -0.1, y_batch + 0.1]).sum().item()
            total += y_batch.size(0)

        acc = correct / total
        print(f"Epoch {epoch}/{epochs} | Loss: {total_loss:.4f} | Accuracy: {acc:.4f}")

In [34]:
#convModel = EmbeddingNetConv1D(7, 12) 

#train_model(convModel, dataloader_all, 100, 0.001)
## simple linear 85%
## simple linear 92% with 16 embedding and 0.01
linearModel = EmbeddingNetLinear(7, 12)

train_model(linearModel, dataloader_all, 50, 0.001)

Epoch 1/50 | Loss: 16777.8389 | Accuracy: 0.0769
Epoch 2/50 | Loss: 16000.5648 | Accuracy: 0.0834
Epoch 3/50 | Loss: 15537.0735 | Accuracy: 0.0864
Epoch 4/50 | Loss: 15197.2043 | Accuracy: 0.0884
Epoch 5/50 | Loss: 14865.3439 | Accuracy: 0.0907
Epoch 6/50 | Loss: 14557.4567 | Accuracy: 0.0925
Epoch 7/50 | Loss: 14408.5031 | Accuracy: 0.0929
Epoch 8/50 | Loss: 14351.4501 | Accuracy: 0.0929
Epoch 9/50 | Loss: 14319.4169 | Accuracy: 0.0929
Epoch 10/50 | Loss: 14298.4596 | Accuracy: 0.0930
Epoch 11/50 | Loss: 14283.7230 | Accuracy: 0.0930
Epoch 12/50 | Loss: 14271.1909 | Accuracy: 0.0930
Epoch 13/50 | Loss: 14261.6693 | Accuracy: 0.0930
Epoch 14/50 | Loss: 14253.0389 | Accuracy: 0.0930
Epoch 15/50 | Loss: 14243.7293 | Accuracy: 0.0930
Epoch 16/50 | Loss: 14238.4403 | Accuracy: 0.0930
Epoch 17/50 | Loss: 14231.4328 | Accuracy: 0.0930
Epoch 18/50 | Loss: 14226.8083 | Accuracy: 0.0930
Epoch 19/50 | Loss: 14221.5830 | Accuracy: 0.0931
Epoch 20/50 | Loss: 14217.5648 | Accuracy: 0.0931
Epoch 21/

## TESTING MODEL

In [7]:
import random
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))      

In [8]:
def get_model_input_based_on_round(cards, round):
    if round == 0:
        input_cards = cards[:2]
    elif round == 1:
        input_cards = cards[:5]
    else:
        input_cards = cards[:7]
    
    # Convert to tensor
    input_tensor = torch.tensor(input_cards, dtype=torch.long)

    # Pad with zeros on the right if needed
    if len(input_tensor) < 7:
        pad_size = 7 - len(input_tensor)
        input_tensor = F.pad(input_tensor, (0, pad_size), value=0)
    else:
        input_tensor = input_tensor[:7]  # Truncate just in case

    return input_tensor.unsqueeze(0)  # shape: (max_len,)
    
    

In [ ]:
def count_intervals(data, interval_size):
    bins = {}
    num_bins = int(1 / interval_size) + 1

    for value in data:
        bin_index = int(value / interval_size)
        bin_start = round(bin_index * interval_size, 10)  # rounding to avoid floating point issues
        bin_end = round(bin_start + interval_size, 10)

        bin_label = f"[{bin_start}, {bin_end})"
        if bin_label not in bins:
            bins[bin_label] = 0
        bins[bin_label] += 1

    return bins



In [37]:
import ultimate
def test_model_with_games(model, num_games = 100):
    model.eval()
    budget = 0
    allBet = 0
    folds = 0
    flops = 0
    rivers = 0
    preflops = 0
    
    # list of predictions
    preds = []
    for i in range(num_games): 
        # generate game (9 cards, played, river, dealer)
        cards = random.sample(num_deck, 9)
        winnings = 0

        # check what round we are in
        round = 0
        round_when_bet = None
        # play  game until it ends
        while True:
            model_input = get_model_input_based_on_round(cards, round)
            
            # Forward pass to get prediction (probability of betting 1)
            with torch.no_grad():
                pred = model(model_input)  # shape: [1, 1]

            pred_prob = pred.item()  # get scalar probability
            preds.append(pred_prob)
            # if prediction > 0.5 => bet, else don't bet
            if pred_prob > 0.5 and round_when_bet is None:
                round_when_bet = round
                break

            round += 1

            if round == 3:
                break

        # calculate winnings
        player_hand = [enumerated_deck[card] for card in cards[0:7]]
        dealer_hand = [enumerated_deck[card] for card in cards[2:]]

        player_combination = ultimate.get_best_hand(player_hand)
        dealer_combination = ultimate.get_best_hand(dealer_hand)

        player_rank = winning_hand_ranks[player_combination]
        dealer_rank = winning_hand_ranks[dealer_combination]

        victor = 0 # 0 = dealer, 1 = player
        
        if player_rank > dealer_rank:
            victor = 1
        elif player_rank == dealer_rank:
            result = ultimate.decider(player_combination, player_hand, 
                                    dealer_combination, dealer_hand)
            if result == "player":
                victor = 1
            elif result == "dealer":	
                victor = 0
            else:
                victor = 2	# need to decide about this
                winnings = 0
        else:
            victor = 0

        # check if ante is valid
        dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
        blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

        # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
        if round_when_bet == 0:
            if victor == 1:
                winnings =  4 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -6 + (1 if not dealer_has_something else 0)
            allBet += 6
            preflops += 1
        elif round_when_bet == 1:
            if victor == 1:
                winnings =  2 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -4 + (1 if not dealer_has_something else 0)
            allBet += 4
            flops += 1
        elif round_when_bet == 2:
            if victor == 1:
                winnings =  1 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -3 + (1 if not dealer_has_something else 0)
            allBet += 3
            rivers += 1
        elif round_when_bet == None:
            winnings = -2
            allBet += 2
            folds += 1
        
 
        budget += winnings
        #print("Winnings: ", winnings)
    
    print("Budget is: ", budget)
    print("Betted: ", allBet)
    print("PreFlops: ", preflops)
    print("Flops:", flops)
    print("Rivers:", rivers)
    print("Folds: ", folds)
    print("Ratio: ", budget / allBet)
    #print(preds)
    interval_size = 0.2
    result = count_intervals(preds, interval_size)
    for interval, count in sorted(result.items()):
        print(f"{interval}: {count}")
    

In [38]:
test_model_with_games(linearModel, 100000)

Budget is:  -15169.5
Betted:  447657
PreFlops:  51953
Flops: 16422
Rivers: 7001
Folds:  24624
Ratio:  -0.033886435373511416
[0.0, 0.2): 18168
[0.2, 0.4): 49381
[0.4, 0.6): 83449
[0.6, 0.8): 23956
[0.8, 1.0): 4718
